In [ ]:
import os
import pandas as pd
import numpy as np
from explain.pipeline import pipeline, process_overall_metrics
from explain.helpers import get_dirs
from tools import train_test

In [ ]:
emb_import_paths = [
#     "/home/arthurcgusmao/Projects/xkbc/algorithms/OpenKE/results/WN11/TransE/1527008113",
    "/home/arthurcgusmao/Projects/xkbc/algorithms/OpenKE/results/FB13/TransE/1527033688",
    "/home/arthurcgusmao/Projects/xkbc/algorithms/OpenKE/results/NELL186/TransE/1526711822",
]

overalls_output_path = "~/WHI_overalls.tsv"

In [ ]:
for path in emb_import_paths:
    pipeline(path)

In [ ]:
metrics_dicts = process_overall_metrics(emb_import_paths)

In [ ]:
pd.DataFrame(metrics_dicts).to_csv(overalls_output_path, sep='\t')

# Run for specific split/relation

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
from explain.Explanator import Explanator
from explain.helpers import ensure_dir

##### example without features #####
# emb_model_path = "/home/arthurcgusmao/Projects/xkbc/algorithms/OpenKE/results/NELL186/TransE/1526711822"
# ground_truth_dataset_path = "/home/arthurcgusmao/Projects/xkbc/algorithms/OpenKE/benchmarks/NELL186/"
# split = "ghat_3nn_2negrate_bern___pra"
# target_relation = "concept:visualartistartform"

##### normal example #####
emb_model_path = "/home/arthurcgusmao/Projects/xkbc/algorithms/OpenKE/results/NELL186/TransE/1526711822"
ground_truth_dataset_path = "/home/arthurcgusmao/Projects/xkbc/algorithms/OpenKE/benchmarks/NELL186/"
split = "ghat_3nn_2negrate_bern___pra"
target_relation = "concept:teamalsoknownas"

expl = Explanator(emb_model_path, ground_truth_dataset_path)

# variables and ensure dir
pra_results_path  = emb_model_path + '/pra_explain/results/'
expl_results_path = emb_model_path + '/pra_explain/results_explained/'
ensure_dir(expl_results_path)
split_path  = os.path.join(pra_results_path,  split)
output_path = os.path.join(expl_results_path, split)
ensure_dir(output_path)

# run functions
if expl.load_data(split_path, target_relation):
    expl.train_global_logit()
    expl.explain_model(output_path=output_path)
    # expl.explain_per_example(output_path)
    res = expl.get_results()
else:
    print "Could not load data."

res

# Debug

In [ ]:
logits_test  = expl.test_x.multiply(expl.model.coef_)

In [ ]:
logits_test.todense()[34]

In [ ]:
res = logits_test.multiply(expl.feature_lens)

In [ ]:
res.sum()

In [ ]:
(res != 0).sum()

In [ ]:
(np.matrix([[1,2], [0,4]]).multiply([1,2])).sum()

In [ ]:
expl.test_x

In [ ]:
expl.test_x.multiply(expl.feature_lens)

In [ ]:
sum_ = np.sum(expl.test_x.multiply(expl.feature_lens)) # sum of all rule lengths for the relation


Debug sparse matrix

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
sm = csr_matrix([[0,0,1,1,0,1], [1,1,1,0,0,24]])

In [ ]:
(sm != 0).multiply([0,0,0,0,5,1000]).sum()

In [ ]:
res = sm.multiply([0,0,0,1,5,0])
res.eliminate_zeros()

In [ ]:
res.todense()

In [ ]:
res

In [ ]:
(res.getnnz(axis=-1) > 0).sum()

In [ ]:
res.getnnz()

In [ ]:
sm.getnnz(axis=-1).sum()

In [ ]:
np.average([1,3,4,5,1,1,1,90,76])

In [ ]:
np.nanmean([1,3,4,5,1,1,1,np.nan,90,76])

In [ ]:
res.todense()

In [ ]:
expl.train_global_logit()

In [ ]:
import os
import pandas as pd

from explain.helpers import ensure_dir, get_dirs
from explain.Explanator import Explanator
import explain.local_functions as lfs


def pipeline_debug(emb_model_path, splits=None):
    """Runs a pipeline for producing explanations with different models for an embedding model.

    Arguments:
    - `emb_model_path`: (string) path to the embedding model directory.
    - `splits`: (list) directory names (inside `/pra_explain/results`) for which the pipeline
                       should be run.
    """
    # read model information
    model_info = pd.read_csv(emb_model_path + '/model_info.tsv', sep='\t')
    ground_truth_dataset_path = './benchmarks/' + model_info['dataset_name'].iloc[0]

    # define directory path variables
    pra_results_path  = emb_model_path + '/pra_explain/results/'
    expl_results_path = emb_model_path + '/pra_explain/results_explained/'
    ensure_dir(expl_results_path)

    # get a list of splits (different feature extractions, e.g., using G and G_hat) to run if not provided
    if splits == None:
        splits = get_dirs(pra_results_path)

    # instantiate Explanator for this model
    expl = Explanator(emb_model_path, ground_truth_dataset_path)

    for split in splits:

        split_path  = os.path.join(pra_results_path,  split)
        output_path = os.path.join(expl_results_path, split)
        ensure_dir(output_path)

        target_relations = get_dirs(split_path) # get a list of target relations
        results = []

        for target_relation in target_relations:
            print("Loading data for `{}`...".format(target_relation))

            if expl.load_data(split_path, target_relation):

                # global logit
                expl.train_global_logit()
                expl.explain_model(output_path=output_path)
                expl.explain_per_example(output_path)
                results.append(expl.get_results())

                # global regression
#                 expl.train_global_regression()
                return expl

expl = pipeline_debug('./results/WN18/TransE/1526710447')

In [ ]:
type(expl.train_heads[1])

In [ ]:
expl.entity2id['06513953']

In [ ]:
expl.test_heads[0]

In [ ]:
df = pd.read_csv('./benchmarks/WN18/entity2id.txt', sep='\t', skiprows=1, names=['name', 'id'], dtype={'id': int, 'name': str}, converters={'id': int, 'name': str})

In [ ]:
df['name'][17071]

In [ ]:
id2name = {row['id']: row['name'] for _, row in df.iterrows()}
name2id = {name: id_ for id_, name in id2name.iteritems()}

In [ ]:
name2id['06513953']

In [ ]:
import numpy as np


In [ ]:
a = np.array([2, 405, 999])

In [ ]:
a > 4

In [ ]:
res = a > 4

In [ ]:
[1 if s else -1 for s in res]

In [ ]:
import os
d1 = os.path.dirname('./results/FB13/')
d2 = os.path.dirname(d1)
d3 = os.path.dirname(d2)
d4 = os.path.dirname(d3)

In [ ]:
d4

In [ ]:
import os
import subprocess
import shlex


def run_command(command):
    process = subprocess.Popen(shlex.split(command), stdout=subprocess.PIPE)
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print output.strip()
    rc = process.poll()
    return rc

In [ ]:
run_command('(cd /home/arthurcgusmao/Projects/xkbc/algorithms/pra/; sbt "runMain edu.cmu.ml.rtw.pra.experiments.ExperimentRunner /home/arthurcgusmao/Projects/xkbc/algorithms/OpenKE/results/FB15k/TransE/1526710056/pra_explain/ g2id_2negrate_bern___pra")')

In [ ]:
l = [1,2,44]

In [ ]:
max(l)

In [ ]:
import os
from tools import generate_g_hat

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

generate_g_hat.run(import_path='./results/WN11/TransE/1527008520', ks=[3,5,7])

In [ ]:
ss = str(None)

In [ ]:
ss

In [ ]:
ss.split(' -#- ')

In [ ]:
s = ''

In [ ]:
for feat in s.split(' -#- '):
    feat_name, value = feat.split(',')


In [ ]:
import pandas as pd
df = pd.DataFrame({'a': [1,2,3], 'b': [3,4,5]})
# df['c'] = [44,99]

In [ ]:
df

In [ ]:
line = "augustus_charles_pugin,male	1	-children-gender-,1.0 -#- -_parents-gender-,1.0"

In [ ]:
ent_pair, label, features = line.split('\t')

In [ ]:
features

In [ ]:
line2 = "augustus_charles_pugin,male	1	"

In [ ]:
ent_pair, label, features = line2.split('\t')

In [ ]:
if features:
    print 'fasidofjpasod'
else:
    print 'works.'

In [ ]:
t = zip([1,2,3], [3,4,5])

In [ ]:
(3,5) in t

In [ ]:
t

In [ ]:
from tools.pra_run import add_triples_without_features

In [ ]:
add_triples_without_features(
    '/home/arthurcgusmao/Projects/xkbc/algorithms/OpenKE/results/FB13/TransE/1527033688/pra_explain/results/g_2negrate_bern___pra/',
    folds_dict
)

In [ ]:
import pandas as pd

In [ ]:
s = pd.Series([1,2,3,97,1])

In [ ]:
np.sum(s)

In [ ]:
print("iofajsdpfo"),
print('faiosdoifj crazyyy')

In [ ]:
from sklearn import

In [ ]:
train_test.read_model_info('/home/arthurcgusmao/Projects/xkbc/algorithms/OpenKE/results/FB13/TransE/1527033688/')

In [ ]:
os.path.abspath('fjaskd/fajsodf/jfaosid/').split(os.sep)

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
z = np.zeros((10,1))

In [ ]:
y = np.array([1,1,0,1,1,1,1,1,0,1]) # 70% 1s

In [ ]:

param_grid = [{
            'l1_ratio': [.1, .5, .7, .9, .95, .99, 1],
            'alpha': [0.01, 0.001, 0.0001],
            'loss': ["log"],
            'penalty': ["elasticnet"],
            'max_iter': [100000],
            'tol': [1e-3],
            'class_weight': ["balanced"],
            'n_jobs': [8]
        }]
gs = GridSearchCV(SGDClassifier(), param_grid)
gs.fit(z, y)
model = gs.best_estimator_

In [ ]:
model.predict([[0], [1]])

In [ ]:
model.predict_proba([[84]])

In [ ]:
model.coef_

In [ ]:
model.intercept_

In [ ]:
from sklearn.dummy import DummyClassifier

clf = DummyClassifier(strategy='prior')
x = np.array([2,3,55,23,4345234]).reshape(-1, 1)
y = np.array([0,1,0,1,1]).reshape(-1, 1)
clf.fit(x, y)

In [ ]:
x2 = np.array([5892374,384,0,277,54]).reshape(-1, 1)

clf.predict(x2)

In [ ]:
X = np.array([]).reshape(4,0)  # ignored
y = [1, -1, 1, 1]
clf = DummyClassifier(strategy='prior')
clf.fit(X, y)

print clf.predict(X)
print clf.predict_proba(X)

In [ ]:
X.shape[:-1] + (1,)

In [ ]:
np.zeros(X.shape[:-1] + (1,))

In [ ]:
import math
import numpy as np
from sklearn.dummy import DummyClassifier

class PriorClassifier(object):
    """Returns a dummy classifier that is used when no features are present in training/validation.
    """
    def __init__(self):
        self.clf = DummyClassifier(strategy='prior')

    def fit(self, X, y):
        self.clf.fit(self._adapt_X(X), y)
        self.coef_ = np.array([[0]])
        proba = self.clf.class_prior_[-1] # get probability of predicting 1, which will be always the same regardless of X
        self.intercept_ = math.log(proba/(1-proba)) # apply logit to probability in order to get the intercept

    def _adapt_X(self, X):
        if X.shape[-1] == 0:
            X = np.zeros(X.shape[:-1] + (1,))
        return X

    def predict(self, X):
        return self.clf.predict(self._adapt_X(X))

    def predict_proba(self, X):
        return self.clf.predict_proba(self._adapt_X(X))
        
    def score(self, X, y, sample_weight=None):
        return self.clf.score(self._adapt_X(X), y, sample_weight)


In [ ]:
X = np.array([]).reshape(4,0)
y = [1, -1, 1, 1]
clf2 = PriorClassifier()
clf2.fit(X, y)

In [ ]:
clf2.predict(np.array([[1]]))

In [ ]:
clf2.predict_proba(np.array([[1]]))[-1][-1]

In [ ]:
clf2.intercept_

In [ ]:
inter = clf2.intercept_; print inter

In [ ]:
1 / (1 + math.exp(-inter))

In [ ]:
import numpy as np
np.average([1,2,3], weights=[100,100,0.01])

In [ ]:
import os
os.path.dirname('/here/now/')

In [ ]:
'/here/now/'.strip('/').split('/')[-1]